**Ghostbusters: Co-occurence of varied class pairs, in the training and test Dataset**

Setup python libraries

1.   TensorFlow
2.   Keras
3.   sklearn
4.   openCV
5.   Random
6.   NumPy

In [11]:
import tensorflow.compat.v2 as tf
from tensorflow.keras import datasets, layers, models
from sklearn import tree
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
import cv2
import numpy as np
import random

Setup Variables




In [12]:
num_of_letters = 26 # @param {type:"integer"}
num_of_train_images = 2500 # @param {type:"integer"}
num_of_train_images_ub = 200 # @param {type:"integer"}
batch_size = 64 # @param {type:"integer"}
biased_ratio = 32 # @param {type:"integer"}
add_unbiased = 0 # @param {type:"integer"}
test_degree = 3 # @param {type:"integer"}
train_degree = 3 # @param {type:"integer"}
max_num = 2 # @param {type:"integer"}
testOrTrain = 1 # @param {type:"integer"}
times_addition = 0 # @param {type:"integer"}
alphabetOrSpots = 1 # @param {type:"integer"} Alphabets - 1, Spots - 0
f1Values = [0]
f1MaxValues = [0] 

Image Generation Functions 

1. basicText - generates a 64, 64 image with the letter and at the locationmentioned as input.
2. genAlphabetImage - takes the int value of the character as the input, generates the location of the alphabet on the canvas randomly.
3. imageWithDots - generate a 64, 64 image with dots in random locations.
4. resizeImageForFour - resizing function used while combining the 4 alphabet images.
5. genImage - generate the image based on the letter list given as input.

In [13]:
def basicText(text, fontScale, x_direction, y_direction):
  image = np.zeros((64,64))
  # font 
  font = cv2.FONT_HERSHEY_SIMPLEX 
  org = (x_direction,y_direction)  
  color = (255, 255)  
  thickness = 2
  image = cv2.putText(image, text, org, font, fontScale,  
                 color, thickness, cv2.LINE_AA, False)
  return image 

In [14]:
def genAlphabetImage(text):
  val = chr(text)
  x_direction = random.randint(0, 32)
  y_direction = random.randint(28,55)
  image = basicText(val, 1, x_direction, y_direction)
  return image

In [15]:
def imageWithDots():
  image = np.zeros((64,64))
  # font 
  color = (255, 255)  
  thickness = 2
  text = '.'
  font = cv2.FONT_HERSHEY_SIMPLEX 
  for i in range(15):
    x_direction = random.randint(0, 63)
    y_direction = random.randint(0, 63)
    org = (x_direction,y_direction)  
    image = cv2.putText(image, text, org, font, 1,  
                  color, thickness, cv2.LINE_AA, False)
  return image 

In [16]:
def resizeImageForFour(image, num):
  new2 = cv2.resize(image,None,fx=1/num, fy=1/num, interpolation = cv2.INTER_CUBIC)
  new2 = new2.reshape(new2.shape[0], new2.shape[1])
  return new2

In [17]:
def genImage(letterList, numOfTimes):
  finalImage = np.zeros((64,64))
  valList = [[0,0],[0,32],[32,0],[32,32]]
  label = np.zeros(num_of_letters)
  temp = ''
  letterList = list(dict.fromkeys(letterList))
  random.shuffle(valList)
  random.shuffle(letterList)
  for j in range(len(letterList)):
    if alphabetOrSpots == 1:
      img = genAlphabetImage((65 + letterList[j]))
    elif alphabetOrSpots == 0:
      img = imageWithDots()
    label[letterList[j]] = 1
    resizeImage = resizeImageForFour(img, 2)
    finalImage[valList[j][0]:valList[j][0] + 32,valList[j][1]:valList[j][1] + 32] = resizeImage
    temp = temp + chr(65 + letterList[j]) + '_'
  temp = '../pics/' + temp + str(numOfTimes)+'_'+str(testOrTrain)+'.jpg'
  cv2.imwrite('{}'.format(str(temp)), finalImage)
  return finalImage, np.array(label), temp

Dataset Generation functions

In [18]:
def imageGenBasedOnMatrix(matrixDegree, arrayVal):
  images = []
  labels = []
  imageNames = []
  letterList = []
  if matrixDegree == 1:
    for i in range(len(arrayVal)):
      letterList.clear()
      letterList.append(i)
      for j in range(int(arrayVal[i])):
        image, label, imageName = genImage(letterList, j)
        images.append(image)
        labels.append(label)
        imageNames.append(imageName)
  elif matrixDegree == 2:
    for i in range(arrayVal.shape[0]):
      for j in range(arrayVal.shape[1]):
        for k in range(int(arrayVal[i][j])):
          letterList = [i, j]
          random.shuffle(letterList)
          image, label, imageName = genImage(letterList, k)
          images.append(image)
          labels.append(label)
          imageNames.append(imageName)
  elif matrixDegree == 3:
    for i in range(arrayVal.shape[0]):
      for j in range(arrayVal.shape[1]):
        for k in range(arrayVal.shape[2]):
          for l in range(int(arrayVal[i][j][k])):
            letterList = [i,j,k]
            random.shuffle(letterList)
            image, label, imageName = genImage(letterList, l)
            images.append(image)
            labels.append(label)
            imageNames.append(imageName)
  elif matrixDegree == 4:
    for i in range(arrayVal.shape[0]):
      for j in range(arrayVal.shape[1]):
        for k in range(arrayVal.shape[2]):
          for m in range(arrayVal.shape[3]):
            for l in range(int(arrayVal[i][j][k][m])):
              letterList = [i,j,k,m]
              random.shuffle(letterList)
              image, label, imageName = genImage(letterList, l)
              images.append(image)
              labels.append(label)
              imageNames.append(imageName)
  return images, labels, imageNames

In [19]:
def imageGenBasedOnCombinations(comboListVal):
  imageList = []
  labelList = []
  imageNameList = []
  for i in range(len(comboListVal)):
    letterList = []
    for j in range(len(list(comboListVal[i]))):
      letterList.append( ord (list(comboListVal[i])[j]) - 65)
    image, label, imageName = genImage(letterList, 1)
    imageList.append(image)
    labelList.append(label)
    imageNameList.append(imageName)
  return imageList, labelList, imageNameList

In [20]:
def imageGenBasedOnLabels(array, maxValue, numOfUComb):
  imageList = []
  labelList = []
  imageNameList = []
  comboListVal = []
  maxVal = np.amax(array)
  print(maxVal)
  while len(comboListVal) < numOfUComb:
    for i in range(num_of_letters):
      for j in range(i+1, num_of_letters):
        if array[i][j] >= (maxVal - maxValue) and len(comboListVal) < numOfUComb and i != j:
          k=random.randint(0, 25)
          temp = [i, j, k]
          if temp not in comboListVal:
            comboListVal.append([i, j, k])
        elif len(comboListVal) > numOfUComb:
          break
  for i in range(len(comboListVal)):
    letterList = np.array(comboListVal[i])
    image, label, imageName = genImage(letterList, 1)
    imageList.append(image)
    labelList.append(label)
    imageNameList.append(imageName)
  return imageList, labelList, imageNameList

In [21]:
def createOddEven(testImages, testLabels):
  trainImages = []
  trainLabels = []
  for i in range(len(testImages)):
    indexes = []
    for j in range(len(testLabels[i])):
      if testLabels[i][j] == 1:
        indexes.append(j)
    if (indexes[0]%2 == 0 and indexes[1]%2 == 0 and indexes[2]%2 == 0) or  (indexes[0]%2 != 0 and indexes[1]%2 != 0 and indexes[2]%2 != 0):
      trainImages.append(testImages[i])
      trainLabels.append(testLabels[i])
  return trainImages, trainLabels

In [22]:
def createRatioBased(ratio, testImages, testLabels):
  trainImages = []
  trainLabels = []
  for i in range(len(testImages)):
    if i%ratio == 0:
      trainImages.append(testImages[i])
      trainLabels.append(testLabels[i])
  print(len(trainImages))
  return trainImages, trainLabels

In [23]:
def dataSetGen():
  global testOrTrain
  if test_degree == 1:
    testArrayVal = np.array([1]*num_of_letters)
  if test_degree == 2:
    testArrayVal = np.array([[1]*num_of_letters]*num_of_letters)
    for i in range(num_of_letters):
      for j in range(num_of_letters):
        if i == j:
          testArrayVal[i][j] = 0
        if i > j:
          testArrayVal[i][j] = 0
  if test_degree == 3:
    testArrayVal = np.array([[[1]*num_of_letters]*num_of_letters]*num_of_letters)
    for i in range(num_of_letters):
      for j in range(num_of_letters):
        for k in range(num_of_letters):
          if i == j or j == k or i == k:
            testArrayVal[i][j][k] = 0
          if i > j or j > k or i > k:
            testArrayVal[i][j][k] = 0
  imageTesting, testLabels, testImages = imageGenBasedOnMatrix(test_degree, testArrayVal)
  TestDataset = create_dataset(testImages, testLabels, False)
  return TestDataset, testLabels, testImages

Data processing pipeline

1. Functions - 
    - parse_function (takes the input of filename and provides post processed and normalised images)
    - create_dataset (used for generating tf dataset for both training and test images)
2. Input required is the Image names and the data labels
3. TensorFlow Dataset

In [24]:
def parse_function(filename, label):
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    # Resize it to fixed shapee 
    image_resized = tf.image.resize(image_decoded, [96, 96])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    image_normalized = tf.image.convert_image_dtype(image_normalized, dtype=tf.float32)
    return image_normalized, label

In [25]:
def create_dataset(filenames, labels, is_training=True):    
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE )
    dataset = dataset.cache()
    if (is_training):
      dataset = dataset.shuffle(20000, reshuffle_each_iteration=True)
    # Batch the data for multiple steps
    dataset = dataset.batch(batch_size)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE) 
    return dataset

Model used - 

1. Input shape - (32, 32, 1)
2. Activation function used - RELU
3. Last layer activation function - Sigmoid

In [26]:
def modelDef():
  model = models.Sequential()
  model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(64,64,1)))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Conv2D(filters=48, kernel_size=(3, 3), activation='relu'))
  model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dropout(.3))
  model.add(layers.Dense(num_of_letters, activation='sigmoid'))
  model.summary()
  return model

Extra functions - 

1. Accuracy calculation functions
2. F1 score calculation functions
3. Correlationn matrix gen functions
4. Distance matrix gen functions

In [27]:
def Sort(sub_li): 
    l = len(sub_li) 
    for i in range(0, l): 
        for j in range(0, l-i-1): 
            if (sub_li[j][1] > sub_li[j + 1][1]): 
                tempo = sub_li[j] 
                sub_li[j]= sub_li[j + 1] 
                sub_li[j + 1]= tempo 
    return sub_li 

In [28]:
def calculate_labels(pred):
  predict_labels = []
  for i in range(0, len(pred)):
    temp = pred[i]
    val = 0.5
    labels = np.zeros(num_of_letters)
    for j in range(0, num_of_letters):
      if(temp[j] >= val):
        labels[j] = 1
    predict_labels.append(labels)
  return predict_labels

In [29]:
def label_to_alphabet(cal):
  predictedLabels = []
  for i in range(0, len(cal)):
    setList = []
    for j in range(0, num_of_letters):
      if(cal[i][j] == 1):
        temp = chr(j + 65)
        setList.append(temp)
    predictedLabels.append(set(setList))
  return predictedLabels

In [30]:
def f1score(y_test, y_predicted, array):
  global f1Values
  global f1MaxValues
  temp_f1 = 0
  temp_recall = 0
  temp_precision = 0
  temp_acc = 0
  f1Val = []
  distVal = []
  for i in range(len(y_test)):
    temp_f1 = temp_f1 + f1_score(y_test[i], y_predicted[i])
    temp_recall = temp_recall + recall_score(y_test[i], y_predicted[i])
    temp_precision = temp_precision + precision_score(y_test[i], y_predicted[i])
    temp_acc = temp_acc + accuracy_score(y_test[i], y_predicted[i])
    dist_temp = imageValueFunc(array, y_test[i])
    dist_temp_max = imageValueMaxFunc(array, y_test[i])
    f1Val.append([f1_score(y_test[i], y_predicted[i]), dist_temp])
    distVal.append([f1_score(y_test[i], y_predicted[i]), dist_temp_max])
  f1score = temp_f1 / len(y_test)
  precision = temp_precision / len(y_test)
  recall = temp_recall / len(y_test)
  acc = temp_acc / len(y_test)
  f1Values = f1Val
  f1MaxValues = distVal
  return acc, f1score, precision, recall

In [31]:
def accuracyCalc(labelVals, predictedLabelVals):
  Sum = 0 
  accValList = []
  labelsList = []
  for i in range(0, len(labelVals)):
    currentLabelValue = labelVals[i]
    currentPredLabelValue = predictedLabelVals[i]
    setIntersection = currentPredLabelValue.intersection(currentLabelValue)
    x = float(len(setIntersection)/len(currentLabelValue))
    #print(len(currentPredLabelValue), len(currentLabelValue))
    if len(currentLabelValue) == len(currentPredLabelValue) and x == 1:
      labelsList.append(currentLabelValue)
      accValList.append(1)
      Sum = Sum + x
    else:
      labelsList.append(currentLabelValue)
      accValList.append(0)
  Accuracy = Sum / len(labelVals)
  return Accuracy, labelsList, accValList

In [32]:
def accuracyFunc(y_pred, y_test, array):
  y_predicted = calculate_labels(y_pred)
  accuracy, f1_score, precision, recall  = f1score(y_test, y_predicted, array)
  y_predicted = label_to_alphabet(y_predicted)
  y_tested = label_to_alphabet(y_test)
  count, labelList, accValList = accuracyCalc(y_tested, y_predicted)
  print(round(count, 3), round(accuracy, 3), round(f1_score, 3), round(precision, 3), round(recall,3))
  return round(f1_score, 3), labelList, accValList

In [33]:
def sumOfListVals(matrixDegree, arrayValue):
  sumVal = 0
  if matrixDegree == 1:
    for i in range(arrayValue.shape[0]):
      sumVal = sumVal + arrayValue[i]
  elif matrixDegree == 2:
    for i in range(arrayValue.shape[0]):
      for j in range(arrayValue.shape[1]):
        sumVal = sumVal + arrayValue[i][j]
  elif matrixDegree == 3:
    for i in range(arrayValue.shape[0]):
      for j in range(arrayValue.shape[1]):
        for k in range(arrayValue.shape[2]):
          sumVal = sumVal + arrayValue[i][j][k]
  return sumVal

In [34]:
def probabilityGenMatrix(matrixDegree, arrayValue):
  probability = []
  combo = [] 
  sumVal = sumOfListVals(matrixDegree, arrayValue)
  if matrixDegree == 1:
    for i in range(num_of_letters):
      temp = arrayValue[i]/(sumVal)
      probability.append(temp)
      combo.append([chr(65+i)])
  elif matrixDegree == 2:
    for i in range(num_of_letters):
      for j in range(num_of_letters):
        temp = arrayValue[i][j]/(sumVal)
        if(i != j):
          probability.append(temp)
          combo.append([chr(65+i), chr(65+j)])
  elif matrixDegree == 3:
    for i in range(arrayValue.shape[0]):
      for j in range(arrayValue.shape[1]):
        for k in range(arrayValue.shape[2]):
          temp = arrayValue[i][j][k]/(sumVal)
          if not (i == j or j == k or i == k):
            probability.append(temp)
            combo.append([chr(65+i), chr(65+j), chr(65+k)])
  return probability, combo

In [35]:
def createComboList(listValues, probability):
  comboListVal = []
  listVal = Sort(listValues)
  for i in range(len(listVal)):
    if (listVal[i][1] <= probability):
      comboListVal.append(listVal[i][2])
  return comboListVal

In [36]:
def probabilityListAndCorrectness(arrayValue, TestDataset, testLabels):
  probabilityList, comboList = probabilityGenMatrix(test_degree, arrayValue)
  ypred = model.predict(TestDataset)
  acc , combList, correctness = accuracyFunc(ypred, testLabels)
  combinedList = []
  print(arrayValue.shape)
  for i in range(len(probabilityList)):
    #print(combList[i], comboList[i])
    correctVal = correctness[combList.index(set(comboList[i]))]
    combinedList.append([correctVal, probabilityList[i], comboList[i]])
  return acc, probabilityList, correctness, combinedList, combList

In [37]:
def numOfDataInc(trainImages, trainLabels, totalLength):
  global testOrTrain
  trainLabelsNoice = trainLabels
  combinationList = label_to_alphabet(trainLabelsNoice)
  numOfVals = int(totalLength/len(trainLabels)) + 1
  for i in range(numOfVals):
    testOrTrain = i + 2
    _, trainLabelsComb, trainImagesComb = imageGenBasedOnCombinations(combinationList)
    for j in range(len(trainLabelsComb)):
      if len(trainImages) == totalLength:
        break
      trainImages.append(trainImagesComb[j])
      trainLabels.append(trainLabelsComb[j])
  return trainImages, trainLabels

In [38]:
def correlationMatrix(trainLabels):
  if test_degree == 2:
    w = np.vstack(trainLabels)
    arr = np.zeros([num_of_letters,num_of_letters])
    for i in range(0, w.shape[0]):
      for j in range(0, w.shape[1]):
        if(w[i][j] == 1):
          for k in range(j+1, w.shape[1]):
            if(w[i][k] == 1):
              arr[j][k] += 1
              arr[k][j] += 1
  elif test_degree == 1:
    arr = np.zeros((num_of_letters))
    for i in range(len(trainLabels)):
      for j in range(len(trainLabels[i])):
        if trainLabels[i][j] == 1:
          arr[j] = arr[j] + 1
  elif test_degree == 3:
    w = np.vstack(trainLabels)
    arr = np.zeros([num_of_letters,num_of_letters,num_of_letters])
    for i in range(0, w.shape[0]):
      for j in range(0, w.shape[1]):
        if(w[i][j] == 1):
          for k in range(j+1, w.shape[1]):
            if(w[i][k] == 1):
              for l in range(k+1, w.shape[1]):
                if(w[i][l] == 1):
                  arr[j][k][l] += 1
                  arr[k][j][l] += 1
                  arr[j][l][k] += 1
                  arr[k][l][j] += 1
                  arr[l][k][j] += 1
                  arr[l][j][k] += 1
  return arr

In [39]:
def correlationmt(labels):
  arr = np.zeros([num_of_letters,num_of_letters])
  for i in range(len(labels)):
    for j in range(num_of_letters):
      if labels[i][j] == 1:
        for k in range(num_of_letters):
          if labels[i][k] == 1:
            arr[j][k] = 1
            arr[k][j] = 1
  return arr

In [40]:
def calculateDist(corr):
  dist = np.zeros([num_of_letters,num_of_letters])
  for i in range(num_of_letters):
    for j in range(num_of_letters):
      if i != j:
        if corr[i][j] == 1 and dist[i][j] == 0:
          dist[i][j] = 1
          dist[j][i] = 1
        elif dist[i][j] == 0:
          for k in range(num_of_letters):
            if corr[k][j] == 1 and k != j and k != i:
              #print("i:", i, "j:", j, "k:", k)
              if corr[i][k] == 1:
                temp1 = 2
                if temp1 < dist[i][j] or dist[i][j] == 0:
                  dist[i][j] = int(temp1)
                  dist[j][i] = int(temp1)
              elif dist[i][k] != 0:
                temp = dist[i][k] + 1
                if temp < dist[i][j] or dist[i][j] == 0:
                  dist[i][j] = int(temp)
                  dist[j][i] = int(temp)
              #print(dist[i][j])
  maxVal = np.amax(dist)
  for i in range(num_of_letters):
    for j in range(num_of_letters):
      if dist[i][j] == 0 and i !=j :
        dist[i][j] = maxVal + 1
      elif i == j:
        dist[i][j] = 0 
  return dist

In [41]:
def imageValueMaxFunc(array, label):
  dist = 0
  for i in range(len(label)):
    if label[i] == 1:
      for j in range(i+1, len(label)):
        if label[j] == 1:
          if array[i][j] == 0:
            if dist < 10:
              dist = 10
          else:
            if dist < array[i][j]:
              dist = array[i][j]
  return int(dist)

In [42]:
def imageValueFunc(array, label):
  dist = 0
  for i in range(len(label)):
    if label[i] == 1:
      for j in range(i+1, len(label)):
        if label[j] == 1:
          if array[i][j] == 0:
            dist = dist + 10
          else:
            dist = dist + array[i][j]
  dist = dist / 3 
  return dist

In [43]:
def convertLabelsToSingleAlphabet(alphabet, trainImages, trainLabels):
  trainLabelsNew = np.zeros((len(trainImages)))
  count = 0 
  for i in range(len(trainImages)):
    if trainLabels[i][alphabet] == 1:
      trainLabelsNew[i] = 1
      count = count + 1
  return trainImages, trainLabelsNew, count

In [44]:
def pickRandom(trainImages, trainLabels, numOfComb):
  trainImagesNew = []
  trainLabelsNew = []
  for i in range(numOfComb):
    temp = random.randint(0, 2599)
    if trainImages[temp] not in trainImagesNew:
      trainImagesNew.append(trainImages[temp])
      trainLabelsNew.append(trainLabels[temp])
  return _, trainLabelsNew, trainImagesNew

In [45]:
def incNumOfImages(trainImages, trainLabels, totalLength):
    numOfVals = int(totalLength/len(trainLabels)) + 1
    for i in range(numOfVals):
        mnist_base.testOrTrain = i + 2
        for j in range(len(trainLabels)):
            combinationList = trainLabels[j]
            _, trainLabelsComb, trainImagesComb = mnist_base.randomLocationGen(combinationList, imagesList, labelsList)
            if len(trainImages) >= totalLength:
                break
            trainImages.append(trainImagesComb)
            trainLabels.append(trainLabelsComb)
    return trainImages, trainLabels

In [46]:
def convertToArray(labelVals):
    labels = []
    for i in range(len(labelVals)):
        label = np.zeros(10)
        for j in range(len(labelVals[i])):
            label[labelVals[i][j]] = 1
        labels.append(label)
    return labels

In [47]:
def convertToNum(labelVals):
    labels = []
    for i in range(len(labelVals)):
        label = []
        for j in range(len(labelVals[i])):
            if labelVals[i][j] == 1:
                label.append(j)
        labels.append(label)
    return labels

In [72]:
def createRollingLabels(numOfObjects, start, end):
    labels = []
    if numOfObjects == 3:
        for i in range(start, end):
            for j in range(i+1, end):
                for k in range(j+1, end):
                    labels.append([i,j,k])
    return labels

In [79]:
def createRollingWindow(windowSize, numOfObjects, testImages, testLabels):
    trainImages = []
    trainLabels = []
    if len(testLabels[0]) == num_of_letters:
        tempLabels = convertToNum(testLabels)
    else:
        tempLabels = testLabels
    for i in range(0, (num_of_letters - windowSize + 1)):
        selectLabels = createRollingLabels(numOfObjects, i , (i + windowSize))
        print(selectLabels)
        for j in range(len(selectLabels)):
            indexVal = tempLabels.index(selectLabels[j])
            trainImages.append(testImages[indexVal])
            trainLabels.append(testLabels[indexVal])
    return trainImages, trainLabels

In [48]:
def createOddEvenNew(testImages, testLabels):
  trainImages = []
  trainLabels = []
  for i in range(len(testImages)):
    indexes = []
    for j in range(len(testLabels[i])):
      indexes.append(testLabels[i][j])
    if (indexes[0]%2 == 0 and indexes[1]%2 == 0 and indexes[2]%2 == 0) or  (indexes[0]%2 != 0 and indexes[1]%2 != 0 and indexes[2]%2 != 0):
      trainImages.append(testImages[i])
      trainLabels.append(testLabels[i])
  return trainImages, trainLabels

In [49]:
def addLessConnectedComb(combinationListVals, trainImages, trainLabels, totalLength):
    numOfVals = int(totalLength/len(combinationListVals)) + 1
    for i in range(numOfVals):
        for j in range(len(combinationListVals)):
            mnist_base.testOrTrain = i + j + 2
            combinationList = combinationListVals[j]
            _, trainLabelsComb, trainImagesComb = mnist_base.randomLocationGen(combinationList, imagesList, labelsList)
            if len(trainImages) >= totalLength:
                break
            trainImages.append(trainImagesComb)
            trainLabels.append(trainLabelsComb)
    return trainImages, trainLabels

In [54]:
def connectednessMeasure(trainLabels, testLabelsVal):
    arrayVal = correlationmt(trainLabels)
    #print(arrayVal)
    connectedness = []
    y_test_num = convertToNum(testLabelsVal)
    for w in range(120):
        '''i = y_test_num[w][0]
        j = y_test_num[w][1]
        k = y_test_num[w][2]'''
        count = 0
        for m in range(10):
            counter = 0
            for i in range(len(y_test_num[w])):
                if arrayVal[y_test_num[w][i]][m] == 1:
                    counter = counter + 1
            '''if (arrayVal[i][m] == 1 and arrayVal[j][m] == 1 and arrayVal[k][m] == 1):
                count = count + 1
            elif (arrayVal[i][m] == 1 and arrayVal[j][m] == 1) or (arrayVal[j][m] == 1 and arrayVal[k][m] == 1) or (arrayVal[i][m] == 1 and arrayVal[k][m] == 1):
                count = count + 2/3
            elif (arrayVal[i][m] == 1 or arrayVal[j][m] == 1 or arrayVal[k][m] == 1):'''
            count = count + counter/len(y_test_num[w])
        connectedness.append([y_test_num[w],round(count/10, 2)])
    return connectedness

In [55]:
def createCombinationList(connectedness, numOfCombinations):
    temp = connectedness
    sortedVal = Sort(temp)
    combList = []
    for i in range(numOfCombinations):
        combList.append(sortedVal[i][0])
    return combList

In [60]:
 def modelLatest():
    IMG_SIZE = (96, 96)
    IMG_SHAPE = IMG_SIZE + (3,)
    base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                   include_top=False)
    inputs = tf.keras.Input(shape=(96, 96, 3))
    x = base_model(inputs, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    return model

19/19 [==============================] - 2s 41ms/step - loss: 0.9791 - accuracy: 0.0000e+00


In [59]:
def createCombinationListRand(connectedness, numOfCombinations):
    combList = []
    tempList = []
    for i in range(numOfCombinations):
        temp = random.randint(0,119)
        if temp not in tempList:
            tempList.append(temp)
            combList.append(connectedness[temp][0])
    return combList

In [69]:
def correlationmtcount(labels):
  arr = np.zeros([num_of_letters,num_of_letters])
  for i in range(len(labels)):
    for j in range(num_of_letters):
      if labels[i][j] == 1:
        for k in range(j + 1, num_of_letters):
          if labels[i][k] == 1:
            arr[j][k] = arr[j][k] + 1
            arr[k][j] = arr[k][j] + 1
  return arr

def connectednessMeasureCount(trainLabels, testLabelsVal):
    arrayVal = correlationmtcount(trainLabels)
    print(arrayVal)
    #print(arrayVal)
    connectedness = []
    y_test_num = convertToNum(testLabelsVal)
    for w in range(120):
        '''i = y_test_num[w][0]
        j = y_test_num[w][1]
        k = y_test_num[w][2]'''
        count = 0
        for m in range(10):
            counter = 0
            for i in range(len(y_test_num[w])):
                if arrayVal[y_test_num[w][i]][m] != 0:
                    counter = counter + arrayVal[y_test_num[w][i]][m]
            '''if (arrayVal[i][m] == 1 and arrayVal[j][m] == 1 and arrayVal[k][m] == 1):
                count = count + 1
            elif (arrayVal[i][m] == 1 and arrayVal[j][m] == 1) or (arrayVal[j][m] == 1 and arrayVal[k][m] == 1) or (arrayVal[i][m] == 1 and arrayVal[k][m] == 1):
                count = count + 2/3
            elif (arrayVal[i][m] == 1 or arrayVal[j][m] == 1 or arrayVal[k][m] == 1):'''
            count = count + counter/len(y_test_num[w])
        connectedness.append([y_test_num[w],round(count/10, 2)])
    return connectedness

In [84]:
random.seed(10)
test_degree=3
alphabetOrSpots = 1
testOrTrain = 0
num_of_letters = 26
_, testLabelsVal, testImagesVal = dataSetGen()

In [85]:
random.seed(100)
test_degree=3
alphabetOrSpots = 1
testOrTrain = 1
num_of_letters = 26
_, trainLabelsVal, trainImagesVal = dataSetGen()

In [51]:
from mnistBase import *

mnist_base = mnistBase()
images, labels = mnist_base.mnistBaseDef('train', False)
mnist_base.imageSize = 96
imagesList, labelsList = mnist_base.convertToList(images, labels)
print(len(images))
#print(labelsList[0])

60000


In [50]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(10)

model = modelLatest()
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

loss0, accuracy0 = model.evaluate(TestDataset)

In [52]:
random.seed(10)
_, trainLabelsVal, trainImagesVal = mnist_base.generateCompleteDataset(imagesList, labelsList) 

In [53]:
random.seed(100)
mnist_base.testOrTrain = 2
_, testLabelsVal, testImagesVal = mnist_base.generateCompleteDataset(imagesList, labelsList) 
testImagesVal, testLabelsVal = incNumOfImages(testImagesVal, testLabelsVal, 1200)
testLabelsVal = convertToArray(testLabelsVal)

In [55]:
random.seed(1000)
biased_ratio = 64
num_of_letters = 10
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
f1scoreVal = []
num_of_train_images = 1000

trainImagesComb, trainLabelsComb = createRatioBased(biased_ratio,trainImagesVal, trainLabelsVal)
#trainImagesComb, trainLabelsComb =createOddEvenNew(trainImagesVal, trainLabelsVal)
trainImages, trainLabels = incNumOfImages(trainImagesComb, trainLabelsComb, num_of_train_images)
trainLabels = convertToArray(trainLabels)
connectedness = connectednessMeasure(trainLabels, testLabelsVal)
combList = createCombinationListRand(connectedness, 15)
print(connectedness)
print(combList)

2
[[[0, 1, 2], 0.37], [[0, 1, 3], 0.3], [[0, 1, 4], 0.3], [[0, 1, 5], 0.2], [[0, 1, 6], 0.2], [[0, 1, 7], 0.2], [[0, 1, 8], 0.2], [[0, 1, 9], 0.2], [[0, 2, 3], 0.37], [[0, 2, 4], 0.37], [[0, 2, 5], 0.27], [[0, 2, 6], 0.27], [[0, 2, 7], 0.27], [[0, 2, 8], 0.27], [[0, 2, 9], 0.27], [[0, 3, 4], 0.3], [[0, 3, 5], 0.2], [[0, 3, 6], 0.2], [[0, 3, 7], 0.2], [[0, 3, 8], 0.2], [[0, 3, 9], 0.2], [[0, 4, 5], 0.2], [[0, 4, 6], 0.2], [[0, 4, 7], 0.2], [[0, 4, 8], 0.2], [[0, 4, 9], 0.2], [[0, 5, 6], 0.1], [[0, 5, 7], 0.1], [[0, 5, 8], 0.1], [[0, 5, 9], 0.1], [[0, 6, 7], 0.1], [[0, 6, 8], 0.1], [[0, 6, 9], 0.1], [[0, 7, 8], 0.1], [[0, 7, 9], 0.1], [[0, 8, 9], 0.1], [[1, 2, 3], 0.37], [[1, 2, 4], 0.37], [[1, 2, 5], 0.27], [[1, 2, 6], 0.27], [[1, 2, 7], 0.27], [[1, 2, 8], 0.27], [[1, 2, 9], 0.27], [[1, 3, 4], 0.3], [[1, 3, 5], 0.2], [[1, 3, 6], 0.2], [[1, 3, 7], 0.2], [[1, 3, 8], 0.2], [[1, 3, 9], 0.2], [[1, 4, 5], 0.2], [[1, 4, 6], 0.2], [[1, 4, 7], 0.2], [[1, 4, 8], 0.2], [[1, 4, 9], 0.2], [[1, 5, 6]

In [80]:
random.seed(1000)
biased_ratio = 8
num_of_letters = 10
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
f1scoreVal = []
num_of_train_images = 1000

#trainImages, trainLabels = createRatioBased(biased_ratio,trainImagesVal, trainLabelsVal)
trainImages, trainLabels = createRollingWindow(5, 3,trainImagesVal, trainLabelsVal)
#trainImagesComb, trainLabelsComb =createOddEvenNew(trainImagesVal, trainLabelsVal)
#= incNumOfImages(trainImagesComb, trainLabelsComb, num_of_train_images)
trainLabels = convertToArray(trainLabels)
connectedness = connectednessMeasure(trainLabels, testLabelsVal)
print(connectedness)

[[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4], [0, 3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
[[1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5], [1, 4, 5], [2, 3, 4], [2, 3, 5], [2, 4, 5], [3, 4, 5]]
[[2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 4, 5], [2, 4, 6], [2, 5, 6], [3, 4, 5], [3, 4, 6], [3, 5, 6], [4, 5, 6]]
[[3, 4, 5], [3, 4, 6], [3, 4, 7], [3, 5, 6], [3, 5, 7], [3, 6, 7], [4, 5, 6], [4, 5, 7], [4, 6, 7], [5, 6, 7]]
[[4, 5, 6], [4, 5, 7], [4, 5, 8], [4, 6, 7], [4, 6, 8], [4, 7, 8], [5, 6, 7], [5, 6, 8], [5, 7, 8], [6, 7, 8]]
[[5, 6, 7], [5, 6, 8], [5, 6, 9], [5, 7, 8], [5, 7, 9], [5, 8, 9], [6, 7, 8], [6, 7, 9], [6, 8, 9], [7, 8, 9]]
[[[0, 1, 2], 0.6], [[0, 1, 3], 0.63], [[0, 1, 4], 0.67], [[0, 1, 5], 0.67], [[0, 1, 6], 0.63], [[0, 1, 7], 0.6], [[0, 1, 8], 0.57], [[0, 1, 9], 0.53], [[0, 2, 3], 0.67], [[0, 2, 4], 0.7], [[0, 2, 5], 0.7], [[0, 2, 6], 0.67], [[0, 2, 7], 0.63], [[0, 2, 8], 0.6], [[0, 2, 9], 0.57], [[0, 3, 4], 0.73], [[0, 3, 5], 0.73], [[0, 3, 6], 0.7

In [47]:
random.seed(1000)
biased_ratio = 1
num_of_letters = 10
array = [500,1000,2000,2500,3000,5000,10000,20000]
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
f1scoreVal = []
precVal = []
recaVal = []
for i in range(len(array)):
    num_of_train_images = array[i]
    
    trainImagesComb, trainLabelsComb = createRatioBased(biased_ratio, trainImagesVal, trainLabelsVal)
    trainImages, trainLabels = incNumOfImages(trainImagesComb, trainLabelsComb, num_of_train_images)
    trainLabels = convertToArray(trainLabels)
    TrainDataset = create_dataset(trainImages, trainLabels)


    model = modelLatest()
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, min_delta=0.0001, restore_best_weights=True)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
    history = model.fit(TrainDataset, epochs=400, validation_data=TestDataset, callbacks=[callback], batch_size = batch_size)

    ypred = model.predict(TestDataset)
    #acc , combList, correctness = accuracyFunc(ypred, testLabelsVal, array)

    temp_f1 = 0
    temp_recall = 0
    temp_precision = 0
    temp_acc = 0
    f1Val = []
    distVal = []
    y_test = testLabelsVal
    y_predicted = calculate_labels(ypred)
    for i in range(len(y_test)):
        temp_f1 = temp_f1 + f1_score(y_test[i], y_predicted[i])
        temp_recall = temp_recall + recall_score(y_test[i], y_predicted[i])
        temp_precision = temp_precision + precision_score(y_test[i], y_predicted[i])
        temp_acc = temp_acc + accuracy_score(y_test[i], y_predicted[i])
        f1Val.append([f1_score(y_test[i], y_predicted[i])])
        distVal.append([f1_score(y_test[i], y_predicted[i])])
    f1score = temp_f1 / len(y_test)
    precision = temp_precision / len(y_test)
    recall = temp_recall / len(y_test)
    acc = temp_acc / len(y_test)
    print(f1score, precision, recall, acc)
    f1scoreVal.append(f1score)
    precVal.append(precision)
    recaVal.append(recall)

#arrayVal = correlationmt(trainLabelsComb)
#array = calculateDist(arrayVal)

120
Epoch 1/400


KeyboardInterrupt: 

In [47]:
print("1")
print(f1scoreVal)
print(precVal)
print(recaVal)

1
[1.0, 0.8324206349206347, 0.8645436507936505, 0.8790476190476187, 0.8645634920634914, 0.9057936507936508, 0.9539285714285715, 0.9623809523809526]
[1.0, 0.9006944444444447, 0.931388888888889, 0.9375000000000002, 0.9173611111111112, 0.9326388888888891, 0.9770833333333334, 0.96875]
[1.0, 0.7972222222222224, 0.8250000000000005, 0.8444444444444447, 0.8361111111111118, 0.8944444444444445, 0.9416666666666668, 0.9611111111111111]


In [48]:
random.seed(1000)
biased_ratio = 1
num_of_letters = 10
array = [500,1000,2000,2500,3000,5000,10000,20000]
f1scoreVal = []
precVal = []
recaVal = []
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
for i in range(len(array)):
    num_of_train_images = array[i]

    trainImagesComb, trainLabelsComb = createOddEvenNew(trainImagesVal, trainLabelsVal)
    trainImages, trainLabels = incNumOfImages(trainImagesComb, trainLabelsComb, num_of_train_images)
    trainLabels = convertToArray(trainLabels)
    TrainDataset = create_dataset(trainImages, trainLabels)

    model = modelLatest()
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, min_delta=0.0001, restore_best_weights=True)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
    history = model.fit(TrainDataset, epochs=400, validation_data=TestDataset, callbacks=[callback], batch_size = batch_size)

    ypred = model.predict(TestDataset)
    #acc , combList, correctness = accuracyFunc(ypred, testLabelsVal, array)

    temp_f1 = 0
    temp_recall = 0
    temp_precision = 0
    temp_acc = 0
    f1Val = []
    distVal = []
    y_test = testLabelsVal
    y_predicted = calculate_labels(ypred)
    for i in range(len(y_test)):
        temp_f1 = temp_f1 + f1_score(y_test[i], y_predicted[i])
        temp_recall = temp_recall + recall_score(y_test[i], y_predicted[i])
        temp_precision = temp_precision + precision_score(y_test[i], y_predicted[i])
        temp_acc = temp_acc + accuracy_score(y_test[i], y_predicted[i])
        f1Val.append([f1_score(y_test[i], y_predicted[i])])
        distVal.append([f1_score(y_test[i], y_predicted[i])])
    f1score = temp_f1 / len(y_test)
    precision = temp_precision / len(y_test)
    recall = temp_recall / len(y_test)
    acc = temp_acc / len(y_test)
    f1scoreVal.append(f1score)
    precVal.append(precision)
    recaVal.append(recall)
    print(f1score, precision, recall, acc, f1_score(y_test, y_predicted, average='micro'))

#arrayVal = correlationmt(trainLabelsComb)
#array = calculateDist(arrayVal)

Epoch 1/400
8/8 [==============================] - 5s 264ms/step - loss: 0.6950 - accuracy: 0.2330 - val_loss: 0.5753 - val_accuracy: 0.4083
Epoch 2/400
8/8 [==============================] - 1s 178ms/step - loss: 0.4398 - accuracy: 0.4320 - val_loss: 0.6097 - val_accuracy: 0.4750
Epoch 3/400
8/8 [==============================] - 1s 179ms/step - loss: 0.2970 - accuracy: 0.5075 - val_loss: 0.6075 - val_accuracy: 0.3833
Epoch 4/400
8/8 [==============================] - 1s 177ms/step - loss: 0.2030 - accuracy: 0.4535 - val_loss: 0.6061 - val_accuracy: 0.3667
Epoch 5/400
8/8 [==============================] - 1s 176ms/step - loss: 0.1185 - accuracy: 0.5048 - val_loss: 0.6117 - val_accuracy: 0.4083
Epoch 6/400
8/8 [==============================] - 1s 176ms/step - loss: 0.0774 - accuracy: 0.5179 - val_loss: 0.6117 - val_accuracy: 0.3667
Epoch 7/400
8/8 [==============================] - 1s 177ms/step - loss: 0.0495 - accuracy: 0.4418 - val_loss: 0.6396 - val_accuracy: 0.4417
Epoch 8/400
8

/home/sidharth/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.1525 0.3 0.1027777777777778 0.7266666666666677 0.18407960199004975
Epoch 1/400
16/16 [==============================] - 7s 207ms/step - loss: 0.6265 - accuracy: 0.3278 - val_loss: 0.6339 - val_accuracy: 0.3583
Epoch 2/400
16/16 [==============================] - 3s 170ms/step - loss: 0.2426 - accuracy: 0.4321 - val_loss: 0.6147 - val_accuracy: 0.3167
Epoch 3/400
16/16 [==============================] - 3s 169ms/step - loss: 0.1086 - accuracy: 0.4724 - val_loss: 0.6491 - val_accuracy: 0.3583
Epoch 4/400
16/16 [==============================] - 3s 168ms/step - loss: 0.0405 - accuracy: 0.4658 - val_loss: 0.6955 - val_accuracy: 0.3500
Epoch 5/400
16/16 [==============================] - 3s 171ms/step - loss: 0.0221 - accuracy: 0.4272 - val_loss: 0.7104 - val_accuracy: 0.3750
Epoch 6/400
16/16 [==============================] - 3s 170ms/step - loss: 0.0144 - accuracy: 0.4714 - val_loss: 0.6780 - val_accuracy: 0.3833
Epoch 7/400
16/16 [==============================] - 3s 167ms/step - loss

0.424722222222222 0.5972222222222222 0.35833333333333345 0.7583333333333332 0.4708029197080292
Epoch 1/400
32/32 [==============================] - 9s 184ms/step - loss: 0.4888 - accuracy: 0.3946 - val_loss: 0.6583 - val_accuracy: 0.4083
Epoch 2/400
32/32 [==============================] - 5s 169ms/step - loss: 0.0739 - accuracy: 0.5628 - val_loss: 0.6941 - val_accuracy: 0.4583
Epoch 3/400
32/32 [==============================] - 5s 169ms/step - loss: 0.0151 - accuracy: 0.4574 - val_loss: 0.7681 - val_accuracy: 0.4333
Epoch 4/400
32/32 [==============================] - 5s 166ms/step - loss: 0.0075 - accuracy: 0.4560 - val_loss: 0.7532 - val_accuracy: 0.4750
Epoch 5/400
32/32 [==============================] - 5s 166ms/step - loss: 0.0031 - accuracy: 0.4824 - val_loss: 0.8095 - val_accuracy: 0.3917
Epoch 6/400
32/32 [==============================] - 5s 168ms/step - loss: 0.0011 - accuracy: 0.4225 - val_loss: 0.8623 - val_accuracy: 0.4500
Epoch 7/400
32/32 [============================

0.44912698412698376 0.6055555555555556 0.3888888888888889 0.7508333333333336 0.4835924006908463
Epoch 1/400
40/40 [==============================] - 11s 183ms/step - loss: 0.4397 - accuracy: 0.4314 - val_loss: 0.6358 - val_accuracy: 0.5500
Epoch 2/400
40/40 [==============================] - 7s 166ms/step - loss: 0.0421 - accuracy: 0.4887 - val_loss: 0.6811 - val_accuracy: 0.5000
Epoch 3/400
40/40 [==============================] - 7s 167ms/step - loss: 0.0115 - accuracy: 0.4954 - val_loss: 0.6829 - val_accuracy: 0.5167
Epoch 4/400
40/40 [==============================] - 7s 168ms/step - loss: 0.0039 - accuracy: 0.4329 - val_loss: 0.7214 - val_accuracy: 0.5000
Epoch 5/400
40/40 [==============================] - 7s 167ms/step - loss: 0.0020 - accuracy: 0.5104 - val_loss: 0.8002 - val_accuracy: 0.4583
Epoch 6/400
40/40 [==============================] - 7s 168ms/step - loss: 0.0061 - accuracy: 0.4269 - val_loss: 0.7506 - val_accuracy: 0.4917
Epoch 7/400
40/40 [==========================

0.4908531746031741 0.6938888888888889 0.40833333333333355 0.7766666666666663 0.5231316725978647
Epoch 1/400
47/47 [==============================] - 13s 182ms/step - loss: 0.3961 - accuracy: 0.4426 - val_loss: 0.6724 - val_accuracy: 0.3667
Epoch 2/400
47/47 [==============================] - 8s 168ms/step - loss: 0.0238 - accuracy: 0.5018 - val_loss: 0.7324 - val_accuracy: 0.4000
Epoch 3/400
47/47 [==============================] - 8s 170ms/step - loss: 0.0074 - accuracy: 0.5371 - val_loss: 0.7424 - val_accuracy: 0.4083
Epoch 4/400
47/47 [==============================] - 8s 168ms/step - loss: 0.0033 - accuracy: 0.4618 - val_loss: 0.8070 - val_accuracy: 0.4417
Epoch 5/400
47/47 [==============================] - 8s 169ms/step - loss: 0.0016 - accuracy: 0.5226 - val_loss: 0.8798 - val_accuracy: 0.4000
Epoch 6/400
47/47 [==============================] - 8s 169ms/step - loss: 8.6784e-04 - accuracy: 0.4013 - val_loss: 0.8435 - val_accuracy: 0.3833
Epoch 7/400
47/47 [======================

0.5071825396825392 0.7123611111111113 0.43055555555555564 0.7791666666666666 0.5391304347826087
Epoch 1/400
79/79 [==============================] - 17s 175ms/step - loss: 0.3099 - accuracy: 0.4477 - val_loss: 0.7577 - val_accuracy: 0.4167
Epoch 2/400
79/79 [==============================] - 13s 168ms/step - loss: 0.0107 - accuracy: 0.4855 - val_loss: 0.8229 - val_accuracy: 0.3917
Epoch 3/400
79/79 [==============================] - 13s 170ms/step - loss: 0.0034 - accuracy: 0.4197 - val_loss: 0.8562 - val_accuracy: 0.3250
Epoch 4/400
79/79 [==============================] - 13s 169ms/step - loss: 0.0026 - accuracy: 0.4085 - val_loss: 0.8748 - val_accuracy: 0.3750
Epoch 5/400
79/79 [==============================] - 14s 174ms/step - loss: 3.2388e-04 - accuracy: 0.3888 - val_loss: 0.9440 - val_accuracy: 0.3500
Epoch 6/400
79/79 [==============================] - 13s 170ms/step - loss: 2.1062e-04 - accuracy: 0.3343 - val_loss: 0.9483 - val_accuracy: 0.3917
Epoch 7/400
79/79 [=============

0.5682936507936504 0.7130555555555559 0.5055555555555554 0.7849999999999998 0.5852090032154341
Epoch 1/400
157/157 [==============================] - 32s 173ms/step - loss: 0.1975 - accuracy: 0.4637 - val_loss: 0.6844 - val_accuracy: 0.3750
Epoch 2/400
157/157 [==============================] - 27s 169ms/step - loss: 0.0070 - accuracy: 0.4342 - val_loss: 0.7566 - val_accuracy: 0.3917
Epoch 3/400
157/157 [==============================] - 27s 170ms/step - loss: 0.0019 - accuracy: 0.3526 - val_loss: 0.7882 - val_accuracy: 0.3500
Epoch 4/400
157/157 [==============================] - 26s 168ms/step - loss: 0.0015 - accuracy: 0.3380 - val_loss: 0.9397 - val_accuracy: 0.3417
Epoch 5/400
157/157 [==============================] - 27s 170ms/step - loss: 4.5595e-04 - accuracy: 0.3008 - val_loss: 0.9371 - val_accuracy: 0.4167
Epoch 6/400
157/157 [==============================] - 27s 169ms/step - loss: 5.6686e-05 - accuracy: 0.3283 - val_loss: 0.9319 - val_accuracy: 0.3500
Epoch 7/400
157/157 [

0.6149206349206345 0.7673611111111113 0.5444444444444445 0.8066666666666668 0.6282051282051281
Epoch 1/400
313/313 [==============================] - 59s 175ms/step - loss: 0.1316 - accuracy: 0.4695 - val_loss: 0.7200 - val_accuracy: 0.4917
Epoch 2/400
313/313 [==============================] - 53s 170ms/step - loss: 0.0013 - accuracy: 0.4115 - val_loss: 0.7497 - val_accuracy: 0.4500
Epoch 3/400
313/313 [==============================] - 53s 171ms/step - loss: 0.0173 - accuracy: 0.3826 - val_loss: 0.8661 - val_accuracy: 0.5083
Epoch 4/400
313/313 [==============================] - 53s 170ms/step - loss: 0.0012 - accuracy: 0.4002 - val_loss: 0.8543 - val_accuracy: 0.4500
Epoch 5/400
313/313 [==============================] - 53s 170ms/step - loss: 2.0900e-04 - accuracy: 0.3462 - val_loss: 0.8776 - val_accuracy: 0.4750
Epoch 6/400
313/313 [==============================] - 53s 169ms/step - loss: 5.6972e-05 - accuracy: 0.3468 - val_loss: 0.9464 - val_accuracy: 0.4583
Epoch 7/400
313/313 [

0.6761309523809518 0.8445833333333336 0.5972222222222219 0.8383333333333339 0.689102564102564


In [49]:
print("odd+even")
print(f1scoreVal)
print(precVal)
print(recaVal)

odd+even
[0.1525, 0.424722222222222, 0.44912698412698376, 0.4908531746031741, 0.5071825396825392, 0.5682936507936504, 0.6149206349206345, 0.6761309523809518]
[0.3, 0.5972222222222222, 0.6055555555555556, 0.6938888888888889, 0.7123611111111113, 0.7130555555555559, 0.7673611111111113, 0.8445833333333336]
[0.1027777777777778, 0.35833333333333345, 0.3888888888888889, 0.40833333333333355, 0.43055555555555564, 0.5055555555555554, 0.5444444444444445, 0.5972222222222219]


In [50]:
random.seed(1000)
biased_ratio = 32
num_of_letters = 10
array = [500,1000,2000,2500,3000,5000,10000]
f1scoreVal = []
precVal = []
recaVal = []
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
for i in range(len(array)):
    num_of_train_images = array[i]

    trainImagesComb, trainLabelsComb = createOddEvenNew(trainImagesVal, trainLabelsVal)
    trainImages, trainLabels = incNumOfImages(trainImagesComb, trainLabelsComb, num_of_train_images)
    trainLabels = convertToArray(trainLabels)
    TrainDataset = create_dataset(trainImages, trainLabels)

    model = modelLatest()
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, min_delta=0.0001, restore_best_weights=True)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
    history = model.fit(TrainDataset, epochs=400, validation_data=TestDataset, callbacks=[callback], batch_size = batch_size)

    ypred = model.predict(TestDataset)
    #acc , combList, correctness = accuracyFunc(ypred, testLabelsVal, array)

    temp_f1 = 0
    temp_recall = 0
    temp_precision = 0
    temp_acc = 0
    f1Val = []
    distVal = []
    y_test = testLabelsVal
    y_predicted = calculate_labels(ypred)
    for i in range(len(y_test)):
        temp_f1 = temp_f1 + f1_score(y_test[i], y_predicted[i])
        temp_recall = temp_recall + recall_score(y_test[i], y_predicted[i])
        temp_precision = temp_precision + precision_score(y_test[i], y_predicted[i])
        temp_acc = temp_acc + accuracy_score(y_test[i], y_predicted[i])
        f1Val.append([f1_score(y_test[i], y_predicted[i])])
        distVal.append([f1_score(y_test[i], y_predicted[i])])
    f1score = temp_f1 / len(y_test)
    precision = temp_precision / len(y_test)
    recall = temp_recall / len(y_test)
    acc = temp_acc / len(y_test)
    f1scoreVal.append(f1score)
    precVal.append(precision)
    recaVal.append(recall)
    print(f1score, precision, recall, acc, f1_score(y_test, y_predicted, average='micro'))

#arrayVal = correlationmt(trainLabelsComb)
#array = calculateDist(arrayVal)

Epoch 1/400
8/8 [==============================] - 6s 271ms/step - loss: 0.6096 - accuracy: 0.2507 - val_loss: 0.5955 - val_accuracy: 0.3250
Epoch 2/400
8/8 [==============================] - 1s 178ms/step - loss: 0.2902 - accuracy: 0.5095 - val_loss: 0.6233 - val_accuracy: 0.4000
Epoch 3/400
8/8 [==============================] - 1s 179ms/step - loss: 0.1759 - accuracy: 0.5314 - val_loss: 0.5777 - val_accuracy: 0.4083
Epoch 4/400
8/8 [==============================] - 1s 178ms/step - loss: 0.0933 - accuracy: 0.4820 - val_loss: 0.6187 - val_accuracy: 0.4583
Epoch 5/400
8/8 [==============================] - 1s 179ms/step - loss: 0.0573 - accuracy: 0.4700 - val_loss: 0.6294 - val_accuracy: 0.4000
Epoch 6/400
8/8 [==============================] - 1s 178ms/step - loss: 0.0346 - accuracy: 0.4140 - val_loss: 0.6384 - val_accuracy: 0.4417
Epoch 7/400
8/8 [==============================] - 1s 182ms/step - loss: 0.0221 - accuracy: 0.4247 - val_loss: 0.6608 - val_accuracy: 0.5083
Epoch 8/400
8

0.4671230158730156 0.6498611111111111 0.4055555555555557 0.7716666666666665 0.5159010600706714
Epoch 1/400
16/16 [==============================] - 7s 211ms/step - loss: 0.5096 - accuracy: 0.3889 - val_loss: 0.6122 - val_accuracy: 0.4333
Epoch 2/400
16/16 [==============================] - 3s 171ms/step - loss: 0.1618 - accuracy: 0.5229 - val_loss: 0.6427 - val_accuracy: 0.4000
Epoch 3/400
16/16 [==============================] - 3s 171ms/step - loss: 0.0623 - accuracy: 0.5198 - val_loss: 0.6731 - val_accuracy: 0.4333
Epoch 4/400
16/16 [==============================] - 3s 170ms/step - loss: 0.0305 - accuracy: 0.5107 - val_loss: 0.7007 - val_accuracy: 0.4917
Epoch 5/400
16/16 [==============================] - 3s 171ms/step - loss: 0.0201 - accuracy: 0.4931 - val_loss: 0.7216 - val_accuracy: 0.4167
Epoch 6/400
16/16 [==============================] - 3s 172ms/step - loss: 0.0101 - accuracy: 0.4247 - val_loss: 0.7540 - val_accuracy: 0.4083
Epoch 7/400
16/16 [============================

0.38742063492063483 0.6023611111111111 0.31111111111111117 0.7583333333333331 0.4357976653696498
Epoch 1/400
32/32 [==============================] - 10s 186ms/step - loss: 0.4463 - accuracy: 0.4680 - val_loss: 0.6467 - val_accuracy: 0.4917
Epoch 2/400
32/32 [==============================] - 5s 166ms/step - loss: 0.0467 - accuracy: 0.5596 - val_loss: 0.7658 - val_accuracy: 0.4250
Epoch 3/400
32/32 [==============================] - 5s 166ms/step - loss: 0.0118 - accuracy: 0.4703 - val_loss: 0.8274 - val_accuracy: 0.4250
Epoch 4/400
32/32 [==============================] - 5s 166ms/step - loss: 0.0060 - accuracy: 0.4625 - val_loss: 0.8337 - val_accuracy: 0.4417
Epoch 5/400
32/32 [==============================] - 5s 166ms/step - loss: 0.0033 - accuracy: 0.4759 - val_loss: 0.8776 - val_accuracy: 0.4833
Epoch 6/400
32/32 [==============================] - 5s 166ms/step - loss: 0.0018 - accuracy: 0.4961 - val_loss: 0.9004 - val_accuracy: 0.4083
Epoch 7/400
32/32 [=========================

0.5030753968253964 0.7611111111111112 0.4055555555555556 0.783333333333333 0.5289855072463768
Epoch 1/400
40/40 [==============================] - 11s 180ms/step - loss: 0.3995 - accuracy: 0.4689 - val_loss: 0.6190 - val_accuracy: 0.4333
Epoch 2/400
40/40 [==============================] - 7s 167ms/step - loss: 0.0322 - accuracy: 0.5136 - val_loss: 0.6547 - val_accuracy: 0.4333
Epoch 3/400
40/40 [==============================] - 7s 167ms/step - loss: 0.0079 - accuracy: 0.4877 - val_loss: 1.0691 - val_accuracy: 0.3000
Epoch 4/400
40/40 [==============================] - 7s 165ms/step - loss: 0.0320 - accuracy: 0.4741 - val_loss: 0.7491 - val_accuracy: 0.4667
Epoch 5/400
40/40 [==============================] - 7s 166ms/step - loss: 0.0020 - accuracy: 0.4612 - val_loss: 0.7851 - val_accuracy: 0.4417
Epoch 6/400
40/40 [==============================] - 7s 166ms/step - loss: 9.0156e-04 - accuracy: 0.4289 - val_loss: 0.8175 - val_accuracy: 0.4250
Epoch 7/400
40/40 [========================

0.5182738095238091 0.7250000000000001 0.4416666666666666 0.7866666666666663 0.554006968641115
Epoch 1/400
47/47 [==============================] - 13s 183ms/step - loss: 0.3621 - accuracy: 0.4627 - val_loss: 0.7401 - val_accuracy: 0.4750
Epoch 2/400
47/47 [==============================] - 8s 170ms/step - loss: 0.0228 - accuracy: 0.4897 - val_loss: 0.8434 - val_accuracy: 0.3917
Epoch 3/400
47/47 [==============================] - 8s 169ms/step - loss: 0.0056 - accuracy: 0.4574 - val_loss: 0.8823 - val_accuracy: 0.3750
Epoch 4/400
47/47 [==============================] - 8s 170ms/step - loss: 0.0021 - accuracy: 0.3969 - val_loss: 0.9232 - val_accuracy: 0.4333
Epoch 5/400
47/47 [==============================] - 8s 172ms/step - loss: 9.3165e-04 - accuracy: 0.4258 - val_loss: 0.9582 - val_accuracy: 0.4250
Epoch 6/400
47/47 [==============================] - 8s 171ms/step - loss: 9.1125e-04 - accuracy: 0.4583 - val_loss: 0.9928 - val_accuracy: 0.3500
Epoch 7/400
47/47 [====================

0.5349206349206344 0.7548611111111111 0.45000000000000007 0.7891666666666666 0.5615251299826689
Epoch 1/400
79/79 [==============================] - 17s 174ms/step - loss: 0.2429 - accuracy: 0.4906 - val_loss: 0.7874 - val_accuracy: 0.4417
Epoch 2/400
79/79 [==============================] - 13s 167ms/step - loss: 0.0104 - accuracy: 0.4818 - val_loss: 0.7999 - val_accuracy: 0.4167
Epoch 3/400
79/79 [==============================] - 13s 169ms/step - loss: 0.0032 - accuracy: 0.4521 - val_loss: 0.8380 - val_accuracy: 0.4250
Epoch 4/400
79/79 [==============================] - 13s 168ms/step - loss: 6.6769e-04 - accuracy: 0.3901 - val_loss: 0.9079 - val_accuracy: 0.3917
Epoch 5/400
79/79 [==============================] - 13s 167ms/step - loss: 2.6992e-04 - accuracy: 0.3696 - val_loss: 0.9496 - val_accuracy: 0.3833
Epoch 6/400
79/79 [==============================] - 13s 168ms/step - loss: 2.4635e-04 - accuracy: 0.4532 - val_loss: 0.9616 - val_accuracy: 0.4167
Epoch 7/400
79/79 [=========

0.5595238095238091 0.7770833333333336 0.47222222222222215 0.7974999999999995 0.5831903945111492
Epoch 1/400
157/157 [==============================] - 32s 172ms/step - loss: 0.1622 - accuracy: 0.4665 - val_loss: 0.7952 - val_accuracy: 0.4667
Epoch 2/400
157/157 [==============================] - 26s 169ms/step - loss: 0.0036 - accuracy: 0.4148 - val_loss: 1.0294 - val_accuracy: 0.4167
Epoch 3/400
157/157 [==============================] - 26s 168ms/step - loss: 0.0050 - accuracy: 0.3634 - val_loss: 0.8799 - val_accuracy: 0.3917
Epoch 4/400
157/157 [==============================] - 26s 167ms/step - loss: 0.0011 - accuracy: 0.4071 - val_loss: 0.8729 - val_accuracy: 0.4000
Epoch 5/400
157/157 [==============================] - 26s 167ms/step - loss: 2.0379e-04 - accuracy: 0.3696 - val_loss: 0.9655 - val_accuracy: 0.3417
Epoch 6/400
157/157 [==============================] - 26s 167ms/step - loss: 0.0020 - accuracy: 0.3657 - val_loss: 0.9196 - val_accuracy: 0.3667
Epoch 7/400
157/157 [===

0.6693253968253966 0.8006944444444447 0.6083333333333332 0.8283333333333331 0.6801242236024845


In [51]:
print("odd+even")
print(f1scoreVal)
print(precVal)
print(recaVal)

odd+even
[0.4671230158730156, 0.38742063492063483, 0.5030753968253964, 0.5182738095238091, 0.5349206349206344, 0.5595238095238091, 0.6693253968253966]
[0.6498611111111111, 0.6023611111111111, 0.7611111111111112, 0.7250000000000001, 0.7548611111111111, 0.7770833333333336, 0.8006944444444447]
[0.4055555555555557, 0.31111111111111117, 0.4055555555555556, 0.4416666666666666, 0.45000000000000007, 0.47222222222222215, 0.6083333333333332]


In [52]:
random.seed(1000)
biased_ratio = 64
num_of_letters = 10
array = [500,1000,2000,2500,3000,5000,10000]
f1scoreVal = []
precVal = []
recaVal = []
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
for i in range(len(array)):
    num_of_train_images = array[i]

    trainImagesComb, trainLabelsComb = createOddEvenNew(trainImagesVal, trainLabelsVal)
    trainImages, trainLabels = incNumOfImages(trainImagesComb, trainLabelsComb, num_of_train_images)
    trainLabels = convertToArray(trainLabels)
    TrainDataset = create_dataset(trainImages, trainLabels)

    model = modelLatest()
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, min_delta=0.0001, restore_best_weights=True)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
    history = model.fit(TrainDataset, epochs=400, validation_data=TestDataset, callbacks=[callback], batch_size = batch_size)

    ypred = model.predict(TestDataset)
    #acc , combList, correctness = accuracyFunc(ypred, testLabelsVal, array)

    temp_f1 = 0
    temp_recall = 0
    temp_precision = 0
    temp_acc = 0
    f1Val = []
    distVal = []
    y_test = testLabelsVal
    y_predicted = calculate_labels(ypred)
    for i in range(len(y_test)):
        temp_f1 = temp_f1 + f1_score(y_test[i], y_predicted[i])
        temp_recall = temp_recall + recall_score(y_test[i], y_predicted[i])
        temp_precision = temp_precision + precision_score(y_test[i], y_predicted[i])
        temp_acc = temp_acc + accuracy_score(y_test[i], y_predicted[i])
        f1Val.append([f1_score(y_test[i], y_predicted[i])])
        distVal.append([f1_score(y_test[i], y_predicted[i])])
    f1score = temp_f1 / len(y_test)
    precision = temp_precision / len(y_test)
    recall = temp_recall / len(y_test)
    acc = temp_acc / len(y_test)
    f1scoreVal.append(f1score)
    precVal.append(precision)
    recaVal.append(recall)
    print(f1score, precision, recall, acc, f1_score(y_test, y_predicted, average='micro'))

#arrayVal = correlationmt(trainLabelsComb)
#array = calculateDist(arrayVal)

Epoch 1/400
8/8 [==============================] - 5s 259ms/step - loss: 0.5581 - accuracy: 0.3695 - val_loss: 0.5878 - val_accuracy: 0.4333
Epoch 2/400
8/8 [==============================] - 1s 181ms/step - loss: 0.2521 - accuracy: 0.4776 - val_loss: 0.6752 - val_accuracy: 0.3500
Epoch 3/400
8/8 [==============================] - 1s 180ms/step - loss: 0.1649 - accuracy: 0.4814 - val_loss: 0.6091 - val_accuracy: 0.4250
Epoch 4/400
8/8 [==============================] - 1s 180ms/step - loss: 0.0865 - accuracy: 0.5041 - val_loss: 0.6661 - val_accuracy: 0.4583
Epoch 5/400
8/8 [==============================] - 1s 181ms/step - loss: 0.0519 - accuracy: 0.5355 - val_loss: 0.6921 - val_accuracy: 0.4333
Epoch 6/400
8/8 [==============================] - 1s 177ms/step - loss: 0.0283 - accuracy: 0.4997 - val_loss: 0.6969 - val_accuracy: 0.4583
Epoch 7/400
8/8 [==============================] - 1s 178ms/step - loss: 0.0183 - accuracy: 0.4459 - val_loss: 0.7003 - val_accuracy: 0.3667
Epoch 8/400
8

0.3419047619047618 0.52875 0.2749999999999998 0.7474999999999998 0.39520958083832336
Epoch 1/400
16/16 [==============================] - 7s 214ms/step - loss: 0.5379 - accuracy: 0.3753 - val_loss: 0.6213 - val_accuracy: 0.4083
Epoch 2/400
16/16 [==============================] - 3s 174ms/step - loss: 0.1601 - accuracy: 0.5167 - val_loss: 0.6634 - val_accuracy: 0.4000
Epoch 3/400
16/16 [==============================] - 3s 175ms/step - loss: 0.0637 - accuracy: 0.5448 - val_loss: 0.7131 - val_accuracy: 0.4750
Epoch 4/400
16/16 [==============================] - 3s 174ms/step - loss: 0.0235 - accuracy: 0.5445 - val_loss: 0.7833 - val_accuracy: 0.4333
Epoch 5/400
16/16 [==============================] - 3s 173ms/step - loss: 0.0146 - accuracy: 0.5083 - val_loss: 0.8219 - val_accuracy: 0.4250
Epoch 6/400
16/16 [==============================] - 3s 175ms/step - loss: 0.0064 - accuracy: 0.4958 - val_loss: 0.8319 - val_accuracy: 0.4917
Epoch 7/400
16/16 [==============================] - 3s 1

0.3811507936507935 0.5755555555555555 0.31388888888888894 0.7566666666666664 0.4362934362934363
Epoch 1/400
32/32 [==============================] - 9s 187ms/step - loss: 0.3932 - accuracy: 0.4923 - val_loss: 0.6480 - val_accuracy: 0.4083
Epoch 2/400
32/32 [==============================] - 5s 170ms/step - loss: 0.0440 - accuracy: 0.5361 - val_loss: 0.7148 - val_accuracy: 0.4583
Epoch 3/400
32/32 [==============================] - 5s 167ms/step - loss: 0.0141 - accuracy: 0.5158 - val_loss: 0.7506 - val_accuracy: 0.4583
Epoch 4/400
32/32 [==============================] - 5s 169ms/step - loss: 0.0039 - accuracy: 0.4933 - val_loss: 0.7684 - val_accuracy: 0.4333
Epoch 5/400
32/32 [==============================] - 5s 167ms/step - loss: 0.0019 - accuracy: 0.4655 - val_loss: 0.8594 - val_accuracy: 0.4250
Epoch 6/400
32/32 [==============================] - 5s 168ms/step - loss: 0.0011 - accuracy: 0.5033 - val_loss: 0.8491 - val_accuracy: 0.4333
Epoch 7/400
32/32 [===========================

0.5403174603174598 0.7493055555555557 0.4555555555555556 0.7866666666666666 0.5616438356164383
Epoch 1/400
40/40 [==============================] - 11s 183ms/step - loss: 0.4398 - accuracy: 0.4640 - val_loss: 0.6814 - val_accuracy: 0.4750
Epoch 2/400
40/40 [==============================] - 7s 167ms/step - loss: 0.0479 - accuracy: 0.5502 - val_loss: 0.7389 - val_accuracy: 0.5167
Epoch 3/400
40/40 [==============================] - 7s 168ms/step - loss: 0.0084 - accuracy: 0.5620 - val_loss: 0.7918 - val_accuracy: 0.4417
Epoch 4/400
40/40 [==============================] - 7s 167ms/step - loss: 0.0040 - accuracy: 0.5601 - val_loss: 0.8009 - val_accuracy: 0.4917
Epoch 5/400
40/40 [==============================] - 7s 168ms/step - loss: 0.0022 - accuracy: 0.5528 - val_loss: 0.8414 - val_accuracy: 0.4417
Epoch 6/400
40/40 [==============================] - 7s 167ms/step - loss: 9.4445e-04 - accuracy: 0.5241 - val_loss: 0.8928 - val_accuracy: 0.4417
Epoch 7/400
40/40 [=======================

0.4830952380952377 0.6979166666666666 0.40000000000000013 0.7766666666666662 0.5179856115107914
Epoch 1/400
47/47 [==============================] - 12s 182ms/step - loss: 0.3637 - accuracy: 0.5123 - val_loss: 0.7676 - val_accuracy: 0.4667
Epoch 2/400
47/47 [==============================] - 8s 170ms/step - loss: 0.0201 - accuracy: 0.5533 - val_loss: 0.8658 - val_accuracy: 0.3750
Epoch 3/400
47/47 [==============================] - 8s 171ms/step - loss: 0.0060 - accuracy: 0.4419 - val_loss: 0.9548 - val_accuracy: 0.4083
Epoch 4/400
47/47 [==============================] - 8s 170ms/step - loss: 0.0029 - accuracy: 0.4377 - val_loss: 0.9609 - val_accuracy: 0.4417
Epoch 5/400
47/47 [==============================] - 8s 170ms/step - loss: 5.4969e-04 - accuracy: 0.5180 - val_loss: 1.0082 - val_accuracy: 0.3750
Epoch 6/400
47/47 [==============================] - 8s 171ms/step - loss: 7.4355e-04 - accuracy: 0.4766 - val_loss: 1.0331 - val_accuracy: 0.3417
Epoch 7/400
47/47 [==================

0.4994047619047614 0.7270833333333335 0.40833333333333344 0.7825000000000003 0.5297297297297298
Epoch 1/400
79/79 [==============================] - 18s 175ms/step - loss: 0.2394 - accuracy: 0.4949 - val_loss: 0.6905 - val_accuracy: 0.3750
Epoch 2/400
79/79 [==============================] - 13s 168ms/step - loss: 0.0075 - accuracy: 0.4607 - val_loss: 0.9627 - val_accuracy: 0.5083
Epoch 3/400
79/79 [==============================] - 13s 167ms/step - loss: 0.0192 - accuracy: 0.4789 - val_loss: 0.7517 - val_accuracy: 0.4667
Epoch 4/400
79/79 [==============================] - 13s 165ms/step - loss: 0.0017 - accuracy: 0.4536 - val_loss: 0.7874 - val_accuracy: 0.4583
Epoch 5/400
79/79 [==============================] - 13s 169ms/step - loss: 0.0011 - accuracy: 0.4790 - val_loss: 0.8733 - val_accuracy: 0.4000
Epoch 6/400
79/79 [==============================] - 13s 168ms/step - loss: 3.5782e-04 - accuracy: 0.4546 - val_loss: 0.8375 - val_accuracy: 0.4333
Epoch 7/400
79/79 [=================

0.6009920634920631 0.7819444444444446 0.5277777777777778 0.8058333333333333 0.6199021207177814
Epoch 1/400
157/157 [==============================] - 31s 171ms/step - loss: 0.1587 - accuracy: 0.4765 - val_loss: 0.7980 - val_accuracy: 0.4250
Epoch 2/400
157/157 [==============================] - 26s 168ms/step - loss: 0.0060 - accuracy: 0.4385 - val_loss: 0.7499 - val_accuracy: 0.4917
Epoch 3/400
157/157 [==============================] - 26s 167ms/step - loss: 0.0010 - accuracy: 0.4125 - val_loss: 0.8614 - val_accuracy: 0.3917
Epoch 4/400
157/157 [==============================] - 26s 168ms/step - loss: 6.0149e-04 - accuracy: 0.3990 - val_loss: 0.8728 - val_accuracy: 0.4500
Epoch 5/400
157/157 [==============================] - 26s 169ms/step - loss: 3.1889e-04 - accuracy: 0.4377 - val_loss: 0.8867 - val_accuracy: 0.3917
Epoch 6/400
157/157 [==============================] - 26s 169ms/step - loss: 6.6563e-04 - accuracy: 0.3926 - val_loss: 0.9415 - val_accuracy: 0.3583
Epoch 7/400
157/1

0.6337301587301584 0.7498611111111114 0.5805555555555556 0.8091666666666668 0.6460587326120556


In [53]:
print("odd+even")
print(f1scoreVal)
print(precVal)
print(recaVal)

odd+even
[0.3419047619047618, 0.3811507936507935, 0.5403174603174598, 0.4830952380952377, 0.4994047619047614, 0.6009920634920631, 0.6337301587301584]
[0.52875, 0.5755555555555555, 0.7493055555555557, 0.6979166666666666, 0.7270833333333335, 0.7819444444444446, 0.7498611111111114]
[0.2749999999999998, 0.31388888888888894, 0.4555555555555556, 0.40000000000000013, 0.40833333333333344, 0.5277777777777778, 0.5805555555555556]


In [79]:
print("32")
print(f1scoreVal)
print(precVal)
print(recaVal)

32
[0.3446031746031743, 0.4317857142857141, 0.4592857142857142, 0.41948412698412685, 0.4803174603174602, 0.4592063492063492, 0.4536507936507936]
[0.44930555555555546, 0.4625000000000001, 0.48958333333333337, 0.43263888888888885, 0.49513888888888896, 0.4694444444444445, 0.476388888888889]
[0.2972222222222221, 0.4166666666666668, 0.44722222222222235, 0.41388888888888903, 0.4722222222222224, 0.45277777777777783, 0.4388888888888891]


In [59]:
random.seed(1000)
biased_ratio = 32
num_of_letters = 10
array = [1000,2000,5000,15000]
f1scoreVal = []
precVal = []
recaVal = []
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
baseLength = 5000
for i in range(len(array)):
    num_of_train_images = array[i]

    #trainImagesComb, trainLabelsComb = createRatioBased(biased_ratio, trainImagesVal, trainLabelsVal)
    trainImagesComb, trainLabelsComb = createOddEvenNew(trainImagesVal, trainLabelsVal)
    trainImages_un, trainLabels_un = incNumOfImages(trainImagesComb, trainLabelsComb, baseLength)
    trainLabels_un = convertToArray(trainLabels_un)
    connectedness = connectednessMeasure(trainLabels_un, testLabelsVal)
    combList = createCombinationListRand(connectedness, 15)
    trainLabels_un = convertToNum(trainLabels_un)
    trainImages, trainLabels = addLessConnectedComb(combList, trainImages_un, trainLabels_un, baseLength + num_of_train_images)
    trainLabels = convertToArray(trainLabels)
    print(len(trainLabels))
    TrainDataset = create_dataset(trainImages, trainLabels)

    model = modelLatest()
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001, restore_best_weights=True)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
    history = model.fit(TrainDataset, epochs=400, validation_data=TestDataset, callbacks=[callback], batch_size = batch_size)

    ypred = model.predict(TestDataset)
    #acc , combList, correctness = accuracyFunc(ypred, testLabelsVal, array)

    temp_f1 = 0
    temp_recall = 0
    temp_precision = 0
    temp_acc = 0
    f1Val = []
    distVal = []
    y_test = testLabelsVal
    y_predicted = calculate_labels(ypred)
    for i in range(len(y_test)):
        temp_f1 = temp_f1 + f1_score(y_test[i], y_predicted[i])
        temp_recall = temp_recall + recall_score(y_test[i], y_predicted[i])
        temp_precision = temp_precision + precision_score(y_test[i], y_predicted[i])
        temp_acc = temp_acc + accuracy_score(y_test[i], y_predicted[i])
        f1Val.append([f1_score(y_test[i], y_predicted[i])])
        distVal.append([f1_score(y_test[i], y_predicted[i])])
    f1score = temp_f1 / len(y_test)
    precision = temp_precision / len(y_test)
    recall = temp_recall / len(y_test)
    acc = temp_acc / len(y_test)
    f1scoreVal.append(f1score)
    precVal.append(precision)
    recaVal.append(recall)
    print(f1score, precision, recall, acc, f1_score(y_test, y_predicted, average='micro'))

#arrayVal = correlationmt(trainLabelsComb)
#array = calculateDist(arrayVal)

6000
Epoch 1/400
94/94 [==============================] - 22s 192ms/step - loss: 0.3480 - accuracy: 0.3686 - val_loss: 0.4521 - val_accuracy: 0.4400
Epoch 2/400
94/94 [==============================] - 18s 189ms/step - loss: 0.0546 - accuracy: 0.4468 - val_loss: 0.4478 - val_accuracy: 0.3983
Epoch 3/400
94/94 [==============================] - 18s 190ms/step - loss: 0.0269 - accuracy: 0.3858 - val_loss: 0.4702 - val_accuracy: 0.3667
Epoch 4/400
94/94 [==============================] - 17s 183ms/step - loss: 0.0135 - accuracy: 0.3685 - val_loss: 0.4342 - val_accuracy: 0.4850
Epoch 5/400
94/94 [==============================] - 17s 185ms/step - loss: 0.0110 - accuracy: 0.3459 - val_loss: 0.5277 - val_accuracy: 0.4267
Epoch 6/400
94/94 [==============================] - 17s 180ms/step - loss: 0.0064 - accuracy: 0.3645 - val_loss: 0.4651 - val_accuracy: 0.4133
Epoch 7/400
94/94 [==============================] - 17s 180ms/step - loss: 0.0018 - accuracy: 0.4129 - val_loss: 0.5032 - val_accu

Epoch 11/400
313/313 [==============================] - 55s 176ms/step - loss: 0.0021 - accuracy: 0.4104 - val_loss: 0.4943 - val_accuracy: 0.4783
Epoch 12/400
313/313 [==============================] - 56s 179ms/step - loss: 0.0053 - accuracy: 0.3799 - val_loss: 0.4653 - val_accuracy: 0.5183
0.8055238095238012 0.8886666666666653 0.7627777777777724 0.8969999999999935 0.8162901307966706


In [60]:
print("odd+even")
print(f1scoreVal)
print(precVal)
print(recaVal)

odd+even
[0.7918650793650703, 0.8321428571428494, 0.8202817460317381, 0.8055238095238012]
[0.8738888888888859, 0.9112499999999987, 0.8703055555555522, 0.8886666666666653]
[0.7455555555555501, 0.7877777777777724, 0.7927777777777719, 0.7627777777777724]


In [ ]:
print(convertToNum(testLabelsVal)[120:240])

In [50]:
random.seed(1000)
biased_ratio = 64
num_of_letters = 10
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
f1scoreVal = []
num_of_train_images = 10000

trainImagesComb, trainLabelsComb = createOddEvenNew(trainImagesVal, trainLabelsVal)
trainImages, trainLabels = incNumOfImages(trainImagesComb, trainLabelsComb, num_of_train_images)
trainLabels = convertToArray(trainLabels)
TrainDataset = create_dataset(trainImages, trainLabels)

model = modelLatest()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001, restore_best_weights=True)
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
history = model.fit(TrainDataset, epochs=400, validation_data=TestDataset, callbacks=[callback], batch_size = batch_size)

ypred = model.predict(TestDataset)
#acc , combList, correctness = accuracyFunc(ypred, testLabelsVal, array)

temp_f1 = 0
temp_recall = 0
temp_precision = 0
temp_acc = 0
f1Val = []
precVal = []
recaVal = []
distVal = []
y_test = testLabelsVal
y_predicted = calculate_labels(ypred)
for i in range(len(y_test)):
    temp_f1 = temp_f1 + f1_score(y_test[i], y_predicted[i])
    temp_recall = temp_recall + recall_score(y_test[i], y_predicted[i])
    temp_precision = temp_precision + precision_score(y_test[i], y_predicted[i])
    temp_acc = temp_acc + accuracy_score(y_test[i], y_predicted[i])
    f1Val.append(f1_score(y_test[i], y_predicted[i]))
    precVal.append(precision_score(y_test[i], y_predicted[i]))
    recaVal.append(recall_score(y_test[i], y_predicted[i]))
    distVal.append([f1_score(y_test[i], y_predicted[i])])
f1score = temp_f1 / len(y_test)
precision = temp_precision / len(y_test)
recall = temp_recall / len(y_test)
acc = temp_acc / len(y_test)
print(f1score, precision, recall, acc, f1_score(y_test, y_predicted, average='micro'))

#arrayVal = correlationmt(trainLabelsComb)
#array = calculateDist(arrayVal)

Epoch 1/400
157/157 [==============================] - 31s 177ms/step - loss: 0.2593 - accuracy: 0.4185 - val_loss: 0.7472 - val_accuracy: 0.3500
Epoch 2/400
157/157 [==============================] - 27s 175ms/step - loss: 0.0038 - accuracy: 0.4091 - val_loss: 0.9166 - val_accuracy: 0.2450
Epoch 3/400
157/157 [==============================] - 27s 173ms/step - loss: 0.0018 - accuracy: 0.3493 - val_loss: 0.8829 - val_accuracy: 0.2967
Epoch 4/400
157/157 [==============================] - 27s 173ms/step - loss: 0.0151 - accuracy: 0.3318 - val_loss: 0.9825 - val_accuracy: 0.3383
Epoch 5/400
157/157 [==============================] - 27s 175ms/step - loss: 4.1976e-04 - accuracy: 0.3154 - val_loss: 0.9595 - val_accuracy: 0.4550
Epoch 6/400
157/157 [==============================] - 27s 173ms/step - loss: 8.5758e-04 - accuracy: 0.3877 - val_loss: 0.9381 - val_accuracy: 0.4217
Epoch 7/400
157/157 [==============================] - 28s 176ms/step - loss: 0.0013 - accuracy: 0.3573 - val_loss: 

/home/sidharth/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.5904285714285669 0.7141666666666662 0.5322222222222224 0.7966666666666653 0.610969387755102


In [51]:
y_test_num = convertToNum(testLabelsVal)

f1Final = np.zeros(120)
precFinal = np.zeros(120)
recaFinal = np.zeros(120)
for i in range(10):
    for j in range(120):
        index = (i*120)+j
        print(j, ((i*120)+j),y_test_num[index])
        f1Final[j] = f1Final[j] + f1Val[index]
        precFinal[j] = precFinal[j] + precVal[index]
        recaFinal[j] = recaFinal[j] + recaVal[index]

0 0 [0, 1, 2]
1 1 [0, 1, 3]
2 2 [0, 1, 4]
3 3 [0, 1, 5]
4 4 [0, 1, 6]
5 5 [0, 1, 7]
6 6 [0, 1, 8]
7 7 [0, 1, 9]
8 8 [0, 2, 3]
9 9 [0, 2, 4]
10 10 [0, 2, 5]
11 11 [0, 2, 6]
12 12 [0, 2, 7]
13 13 [0, 2, 8]
14 14 [0, 2, 9]
15 15 [0, 3, 4]
16 16 [0, 3, 5]
17 17 [0, 3, 6]
18 18 [0, 3, 7]
19 19 [0, 3, 8]
20 20 [0, 3, 9]
21 21 [0, 4, 5]
22 22 [0, 4, 6]
23 23 [0, 4, 7]
24 24 [0, 4, 8]
25 25 [0, 4, 9]
26 26 [0, 5, 6]
27 27 [0, 5, 7]
28 28 [0, 5, 8]
29 29 [0, 5, 9]
30 30 [0, 6, 7]
31 31 [0, 6, 8]
32 32 [0, 6, 9]
33 33 [0, 7, 8]
34 34 [0, 7, 9]
35 35 [0, 8, 9]
36 36 [1, 2, 3]
37 37 [1, 2, 4]
38 38 [1, 2, 5]
39 39 [1, 2, 6]
40 40 [1, 2, 7]
41 41 [1, 2, 8]
42 42 [1, 2, 9]
43 43 [1, 3, 4]
44 44 [1, 3, 5]
45 45 [1, 3, 6]
46 46 [1, 3, 7]
47 47 [1, 3, 8]
48 48 [1, 3, 9]
49 49 [1, 4, 5]
50 50 [1, 4, 6]
51 51 [1, 4, 7]
52 52 [1, 4, 8]
53 53 [1, 4, 9]
54 54 [1, 5, 6]
55 55 [1, 5, 7]
56 56 [1, 5, 8]
57 57 [1, 5, 9]
58 58 [1, 6, 7]
59 59 [1, 6, 8]
60 60 [1, 6, 9]
61 61 [1, 7, 8]
62 62 [1, 7, 9]
63 63 [1, 8,

20 740 [0, 3, 9]
21 741 [0, 4, 5]
22 742 [0, 4, 6]
23 743 [0, 4, 7]
24 744 [0, 4, 8]
25 745 [0, 4, 9]
26 746 [0, 5, 6]
27 747 [0, 5, 7]
28 748 [0, 5, 8]
29 749 [0, 5, 9]
30 750 [0, 6, 7]
31 751 [0, 6, 8]
32 752 [0, 6, 9]
33 753 [0, 7, 8]
34 754 [0, 7, 9]
35 755 [0, 8, 9]
36 756 [1, 2, 3]
37 757 [1, 2, 4]
38 758 [1, 2, 5]
39 759 [1, 2, 6]
40 760 [1, 2, 7]
41 761 [1, 2, 8]
42 762 [1, 2, 9]
43 763 [1, 3, 4]
44 764 [1, 3, 5]
45 765 [1, 3, 6]
46 766 [1, 3, 7]
47 767 [1, 3, 8]
48 768 [1, 3, 9]
49 769 [1, 4, 5]
50 770 [1, 4, 6]
51 771 [1, 4, 7]
52 772 [1, 4, 8]
53 773 [1, 4, 9]
54 774 [1, 5, 6]
55 775 [1, 5, 7]
56 776 [1, 5, 8]
57 777 [1, 5, 9]
58 778 [1, 6, 7]
59 779 [1, 6, 8]
60 780 [1, 6, 9]
61 781 [1, 7, 8]
62 782 [1, 7, 9]
63 783 [1, 8, 9]
64 784 [2, 3, 4]
65 785 [2, 3, 5]
66 786 [2, 3, 6]
67 787 [2, 3, 7]
68 788 [2, 3, 8]
69 789 [2, 3, 9]
70 790 [2, 4, 5]
71 791 [2, 4, 6]
72 792 [2, 4, 7]
73 793 [2, 4, 8]
74 794 [2, 4, 9]
75 795 [2, 5, 6]
76 796 [2, 5, 7]
77 797 [2, 5, 8]
78 798 [2, 5, 

In [52]:
for i in range(len(f1Final)):
    print(f1Final[i]/10, precFinal[i]/10, recaFinal[i]/10)

0.33999999999999997 0.5 0.26666666666666666
0.6133333333333334 0.7333333333333333 0.5333333333333332
0.5599999999999999 0.7 0.4666666666666668
0.6333333333333333 0.8333333333333333 0.5333333333333332
0.6933333333333335 0.7333333333333334 0.6666666666666667
0.72 0.8 0.6666666666666666
0.47333333333333333 0.6333333333333333 0.4
0.5933333333333334 0.9333333333333332 0.4666666666666666
0.5533333333333333 0.7333333333333333 0.4666666666666666
0.9333333333333332 0.9333333333333332 0.9333333333333332
0.5466666666666666 0.6666666666666666 0.4666666666666666
1.0 1.0 1.0
0.49333333333333335 0.5333333333333334 0.4666666666666667
0.9199999999999999 1.0 0.8666666666666666
0.4666666666666666 0.6666666666666666 0.4
0.4999999999999999 0.7 0.39999999999999997
0.52 0.8 0.4
0.72 0.8 0.6666666666666667
0.6133333333333333 0.7333333333333333 0.5333333333333333
0.5133333333333333 0.6333333333333333 0.4666666666666666
0.45999999999999996 0.6 0.4
0.58 0.8 0.4666666666666667
1.0 1.0 1.0
0.64 0.8 0.5333333333333

In [69]:
random.seed(1000)
biased_ratio = 64
num_of_letters = 10
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
f1scoreVal = []
num_of_train_images = 1000

trainImagesComb, trainLabelsComb = createRatioBased(biased_ratio,trainImagesVal, trainLabelsVal)
#trainImagesComb, trainLabelsComb =createOddEvenNew(trainImagesVal, trainLabelsVal)
trainImages, trainLabels = incNumOfImages(trainImagesComb, trainLabelsComb, num_of_train_images)
trainLabels = convertToArray(trainLabels)
connectedness = connectednessMeasure(trainLabels, testLabelsVal)
combList = createCombinationList(connectedness, 20)
print(combList)

2
[[5, 6, 7], [5, 6, 8], [5, 6, 9], [5, 7, 8], [5, 7, 9], [5, 8, 9], [6, 7, 8], [6, 7, 9], [6, 8, 9], [7, 8, 9], [0, 5, 6], [0, 5, 7], [0, 5, 8], [0, 5, 9], [0, 6, 7], [0, 6, 8], [0, 6, 9], [0, 7, 8], [0, 7, 9], [0, 8, 9]]


In [52]:
random.seed(1000)
biased_ratio = 1
num_of_letters = 10
TestDataset = create_dataset(testImagesVal, testLabelsVal, False)
f1scoreVal = []
num_of_train_images = 1000

trainImagesComb, trainLabelsComb = createRatioBased(biased_ratio, trainImagesVal, trainLabelsVal)
trainImages, trainLabels = incNumOfImages(trainImagesComb, trainLabelsComb, num_of_train_images)
trainLabels = convertToArray(trainLabels)
arrayVal = correlationmt(trainLabels)
y_test_num = convertToNum(testLabelsVal)
for w in range(120):
    i = y_test_num[w][0]
    j = y_test_num[w][1]
    k = y_test_num[w][2]
    corr = np.corrcoef(arrayVal[i], arrayVal[j])[0][1]
    cor1 = np.corrcoef(arrayVal[i], arrayVal[k])[0][1]
    cor2 = np.corrcoef(arrayVal[k], arrayVal[j])[0][1]
    abscorr = np.absolute(corr) + np.absolute(cor1) + np.absolute(cor2)
    maxcorr = max(corr, cor1, cor2)
    mincorr = min(corr, cor1, cor2)
    if (np.absolute(maxcorr) >= np.absolute(mincorr)):
        tempcorr = maxcorr
    else:
        tempcorr = (mincorr)
    maxmincorr = abs(corr-cor1) + abs(cor1-cor2) + abs(cor2-corr)
    print(corr, cor1, cor2)

120
-0.11111111111111102 -0.11111111111111102 -0.11111111111111102
-0.11111111111111102 -0.11111111111111105 -0.11111111111111105
-0.11111111111111102 -0.11111111111111105 -0.11111111111111105
-0.11111111111111102 -0.11111111111111109 -0.11111111111111109
-0.11111111111111102 -0.11111111111111109 -0.11111111111111109
-0.11111111111111102 -0.11111111111111112 -0.11111111111111112
-0.11111111111111102 -0.11111111111111112 -0.11111111111111112
-0.11111111111111102 -0.11111111111111112 -0.11111111111111112
-0.11111111111111102 -0.11111111111111105 -0.11111111111111105
-0.11111111111111102 -0.11111111111111105 -0.11111111111111105
-0.11111111111111102 -0.11111111111111109 -0.11111111111111109
-0.11111111111111102 -0.11111111111111109 -0.11111111111111109
-0.11111111111111102 -0.11111111111111112 -0.11111111111111112
-0.11111111111111102 -0.11111111111111112 -0.11111111111111112
-0.11111111111111102 -0.11111111111111112 -0.11111111111111112
-0.11111111111111105 -0.11111111111111105 -0.111111

In [50]:
arrayVal = correlationmt(trainLabels)
for i in range(num_of_letters):
    for j in range(num_of_letters):
        print(i, j ,np.corrcoef(arrayVal[i], arrayVal[j])[0][1])

0 1 2 0.33333333333333304 -0.11111111111111102 -0.11111111111111102
0 1 3 0.3333333333333331 -0.11111111111111102 -0.11111111111111105
0 1 4 0.3333333333333331 -0.11111111111111102 -0.11111111111111105
0 1 5 0.33333333333333315 -0.11111111111111102 -0.11111111111111109
0 1 6 0.33333333333333315 -0.11111111111111102 -0.11111111111111109
0 1 7 0.33333333333333315 -0.11111111111111102 -0.11111111111111112
0 1 8 0.33333333333333315 -0.11111111111111102 -0.11111111111111112
0 1 9 0.33333333333333315 -0.11111111111111102 -0.11111111111111112
0 2 1 0.33333333333333304 -0.11111111111111102 -0.11111111111111102
0 2 3 0.3333333333333331 -0.11111111111111102 -0.11111111111111105
0 2 4 0.3333333333333331 -0.11111111111111102 -0.11111111111111105
0 2 5 0.33333333333333315 -0.11111111111111102 -0.11111111111111109
0 2 6 0.33333333333333315 -0.11111111111111102 -0.11111111111111109
0 2 7 0.33333333333333315 -0.11111111111111102 -0.11111111111111112
0 2 8 0.33333333333333315 -0.11111111111111102 -0.11

4 1 5 0.3333333333333332 -0.11111111111111105 -0.11111111111111109
4 1 6 0.3333333333333332 -0.11111111111111105 -0.11111111111111109
4 1 7 0.3333333333333332 -0.11111111111111105 -0.11111111111111112
4 1 8 0.3333333333333332 -0.11111111111111105 -0.11111111111111112
4 1 9 0.3333333333333332 -0.11111111111111105 -0.11111111111111112
4 2 0 0.33333333333333315 -0.11111111111111105 -0.11111111111111105
4 2 1 0.33333333333333315 -0.11111111111111105 -0.11111111111111105
4 2 3 0.33333333333333315 -0.11111111111111105 -0.11111111111111105
4 2 5 0.3333333333333332 -0.11111111111111105 -0.11111111111111109
4 2 6 0.3333333333333332 -0.11111111111111105 -0.11111111111111109
4 2 7 0.3333333333333332 -0.11111111111111105 -0.11111111111111112
4 2 8 0.3333333333333332 -0.11111111111111105 -0.11111111111111112
4 2 9 0.3333333333333332 -0.11111111111111105 -0.11111111111111112
4 3 0 0.33333333333333315 -0.11111111111111105 -0.11111111111111105
4 3 1 0.33333333333333315 -0.11111111111111105 -0.11111111

8 0 7 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 0 9 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 1 0 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 1 2 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 1 3 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 1 4 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 1 5 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 1 6 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 1 7 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 1 9 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 2 0 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 2 1 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 2 3 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 2 4 0.33333333333333337 -0.11111111111111112 -0.11111111111111112
8 2 5 0.33333333333333337 -0.11111111111111112 -

In [68]:
biased_ratio = 1
num_of_letters = 5000
trainImagesComb, trainLabelsComb = createRatioBased(biased_ratio, trainImagesVal, trainLabelsVal)
trainImages, trainLabels = incNumOfImages(trainImagesComb, trainLabelsComb, num_of_train_images)
trainLabels = convertToArray(trainLabels)
print(trainImagesComb[:4])

120
['../pics/0_1_2_1.jpg', '../pics/0_1_3_1.jpg', '../pics/0_1_4_1.jpg', '../pics/0_1_5_1.jpg']


In [69]:
countA = 0
countZ = 0
for i in range(len(trainLabels)):
    if trainLabels[i][0] == 1:
        countA = countA + 1
    if trainLabels[i][9] == 1:
        countZ = countZ + 1
print(countA, countZ)

756 746


In [57]:
arrayVal = correlationmt(trainLabels)
for i in range(num_of_letters):
    for j in range(num_of_letters):
        print(i, j ,np.corrcoef(arrayVal[i], arrayVal[j])[0][1])

0 0 1.0
0 1 0.3749999999999999
0 2 0.10206207261596577
0 3 0.37499999999999994
0 4 0.3749999999999999
0 5 nan
0 6 nan
0 7 nan
0 8 nan
0 9 nan
1 0 0.3749999999999999
1 1 1.0
1 2 0.10206207261596577
1 3 0.37499999999999994
1 4 0.3749999999999999
1 5 nan
1 6 nan
1 7 nan
1 8 nan
1 9 nan
2 0 0.10206207261596575
2 1 0.10206207261596575
2 2 1.0
2 3 0.10206207261596575
2 4 0.10206207261596575
2 5 nan
2 6 nan
2 7 nan
2 8 nan
2 9 nan
3 0 0.37499999999999994
3 1 0.37499999999999994
3 2 0.10206207261596577
3 3 1.0
3 4 0.37499999999999994
3 5 nan
3 6 nan
3 7 nan
3 8 nan
3 9 nan
4 0 0.3749999999999999
4 1 0.3749999999999999
4 2 0.10206207261596577
4 3 0.37499999999999994
4 4 1.0
4 5 nan
4 6 nan
4 7 nan
4 8 nan
4 9 nan
5 0 nan
5 1 nan
5 2 nan
5 3 nan
5 4 nan
5 5 nan
5 6 nan
5 7 nan
5 8 nan
5 9 nan
6 0 nan
6 1 nan
6 2 nan
6 3 nan
6 4 nan
6 5 nan
6 6 nan
6 7 nan
6 8 nan
6 9 nan
7 0 nan
7 1 nan
7 2 nan
7 3 nan
7 4 nan
7 5 nan
7 6 nan
7 7 nan
7 8 nan
7 9 nan
8 0 nan
8 1 nan
8 2 nan
8 3 nan
8 4 nan
8 5 na

In [46]:
y_test_num = convertToNum(testLabelsVal) 
cols = num_of_letters
rows = num_of_letters
f1matrix = np.zeros((cols, rows))
accmatrix = np.zeros((cols, rows))
precisionMat = np.zeros((cols, rows))
recallMat = np.zeros((cols, rows))
count = np.zeros((cols, rows))
for i in range(len(y_test_num)):
    for j in range(len(y_test_num[i])):
        for k in range(j, len(y_test_num[i])):
            t = y_test_num[i][j]
            w = y_test_num[i][k] 
            if y_test_num[i][j] != y_test_num[i][k]:
                if y_predicted[i][t] == 1 and y_predicted[i][w] == 1:
                    f1Value = 1
                elif y_predicted[i][t] == 0 and y_predicted[i][w] == 0:
                    f1Value = 0
                elif (y_predicted[i][t] == 1 and y_predicted[i][w] == 0) or (y_predicted[i][t] == 0 and y_predicted[i][w] == 1):
                    f1Value = 0.5
                accmatrix[t][w] = accmatrix[t][w] + f1Value
                f1matrix[t][w] = f1matrix[t][w] + f1Val[i]
                precisionMat[t][w] = precisionMat[t][w]  + precVal[i]
                count[t][w] = count[t][w] + 1
                recallMat[t][w]  = recallMat[t][w] + recaVal[i]
                f1matrix[w][t] = f1matrix[w][t] + f1Val[i]
                accmatrix[w][t] = accmatrix[w][t] + f1Value
                precisionMat[w][t] = precisionMat[w][t]  + precVal[i]
                count[w][t] = count[w][t] + 1
                recallMat[w][t]  = recallMat[w][t] + recaVal[i]